# **Praktikum 2 - Generator Teks dengan RNN**

### **Setup**

**Import TensorFlow**

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

**Download Dataset Shakespeare**

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


**Load Data**

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


### **Text Processing**

**Text Vectorize**

In [6]:
example_texts = ['abcdefg', 'xyz'] # converts a string to a numeric representation, the text will be split into tokens first
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup( # create tf.keras.layers.StringLookup layer
vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars) # converting token to id
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup( # inverts this representation using tf.keras.layers.StringLookup
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids) # converts back the characters from the ID vector, and returns them as tf.RaggedTensor characters
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy() # use tf.strings.reduce_join to rejoin characters into a string

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### **Prediction**

**Create Training Sets and Targets**

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) # divide text into sequence examples
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # using batch method to converts these individual characters to the desired size sequence.

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5): # combines the tokens back into a string
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence): # takes a sequence as input, duplicates it, and shifts it to align the input and labels for each time step
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Create Training Batches**

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### **Create Model**

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model): # defines the model as a subclass ofkeras.Model
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### **Model Test**

In [27]:
for input_example_batch, target_example_batch in dataset.take(1): # check the shape of the output
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary() # summarize the model

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # try for the first example in the batch
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [30]:
sampled_indices # next character index prediction

array([31, 65, 11, 63, 22, 31, 23, 21, 31, 57, 59, 37, 65,  3, 34, 35, 31,
       54, 65, 13, 60, 36,  1, 57, 44, 56, 58, 53,  2, 11, 20,  2, 58,  9,
       23,  2, 35, 35, 61, 58, 63, 38, 48, 42, 45,  4, 11, 56, 56, 18,  9,
        6, 62,  2, 17,  9,  2, 16, 35,  3, 28, 32, 32, 52, 10, 48, 42, 45,
       15,  8, 20, 30, 51, 11, 56,  0, 65,  7, 65, 35, 62, 64, 46, 64, 63,
       63, 65, 60, 48, 21, 53, 35, 23,  9, 62, 33, 35, 22, 23,  8])

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy()) # see the text predicted by this untrained model
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"Bohemia\nYou take my lord, I'll give him my commission\nTo let him there a month behind the gest\nPrefi"

Next Char Predictions:
 b"Rz:xIRJHRrtXz!UVRoz?uW\nreqsn :G s.J VVvsxYicf$:qqE.'w D. CV!OSSm3icfB-GQl:q[UNK]z,zVwygyxxzuiHnVJ.wTVIJ-"


### **Train Model**

**Additional Optimizer and Loss Function**

In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1887736, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy() # checks that the exponential of the average loss should be approximately equal to the vocabulary size

65.94187

In [35]:
model.compile(optimizer='adam', loss=loss) # configure the training procedure using the tf.keras.Model.compile method

**Configure Checkpoints**

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Carry out the Training Process**

In [37]:
EPOCHS = 20

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 16s 62ms/step - loss: 2.7115
Epoch 2/20
172/172 [==============================] - 12s 61ms/step - loss: 1.9847
Epoch 3/20
172/172 [==============================] - 13s 59ms/step - loss: 1.7102
Epoch 4/20
172/172 [==============================] - 11s 56ms/step - loss: 1.5496
Epoch 5/20
172/172 [==============================] - 12s 58ms/step - loss: 1.4502
Epoch 6/20
172/172 [==============================] - 13s 61ms/step - loss: 1.3816
Epoch 7/20
172/172 [==============================] - 13s 62ms/step - loss: 1.3294
Epoch 8/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2846
Epoch 9/20
172/172 [==============================] - 13s 57ms/step - loss: 1.2436
Epoch 10/20
172/172 [==============================] - 12s 57ms/step - loss: 1.2036
Epoch 11/20
172/172 [==============================] - 11s 58ms/step - loss: 1.1653
Epoch 12/20
172/172 [==============================] - 12s 59ms/step - loss: 1.1243
E

### **Text Generate**

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The foul wings affords not her then small counsel,
To satisfy your form and noblehes.

CLEOMENES:
Sir, the best father's son, anon of you;
For thou wilt-wicked action lends him hence!
Roweless brave better by the heart, to show yourselves;
Laments as one deigh. Right you him my friends be too resire
Stands again; because I raiment and
The place of your mother in the encounter.

SLY:
Mark'd your highness prove a long such pporising flass,
From forth thou thoughts to the grace, where I must not,
For down as I can: dune my nay, but doubtful
Business in thy death, nor he did look on;
And let your Romeo will be cold.

GREMIO:
Why, there too! thou hast calls' a brother
And that he did; as he hath baught a tide
To fly, if I die, for all the western sleep,
Hooding your conference, which never was it,
Or not a joint might speak--
Much hope it is.

First Citizen:
Our dear son, sir; for this was the farther that
He shall not.

FRIAR LAURENCE:
So, O, thou citizens, but he shall stay
not qui

In [42]:
start = time.time() # the model produces 5 outputs in almost the same time as it took to produce 1 output previously
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nWell, all festerdanning, of these knights. Worthy help it,\nYo, she lays honesty.\n\nGRUMIO:\nWhy, then less heartish braw out of your got.\nThus, I throw my face, my Romeo is im.\n\nGLOUCESTER:\n\nCLARENCE:\n\nBold:\nWelcome to my bossol.\n\nBRUTUS:\nIs the wondrous beat-bed, thus' I believe thee,\nWhen I was born in Henry's nest,\nOr weeding bones and what a thing\nThat word, respective likewise hath embraced,\nAnd safely hither to use me from his city,\nAnd he shull dissemble with free and grown wombs\nTo see our good companion. So, now I remain,\nHave stoop themselves; and therein fair sustier trush\nAnd never beating for this aftering, and I'll venve\nyou so seem'd, away; for once again to this sword;\nIt calls not Kate; behold him first, thou thereby\nfaced that which you in a marbly take them hope.\nHere is the case put in this sweet books,\nNor so severe flaunts, as to the entertainms.\nWe are lecius.\n\nSICINIUS:\nGo before, be it leave:\nHow doth the cou

### **Export Generator Model**

In [43]:
tf.saved_model.save(one_step_model, 'one_step') # saves the model one step, allowing to use it wherever tf.saved_model is received
one_step_reloaded = tf.saved_model.load('one_step')

In [44]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
This being sport my breaking that's mictry, but I
would over leave to visit me ups!

WARWICK:
Neith


### **Run the Model and calculate the loss with tf.GradientTape**

In [45]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

This custom training step allows for more control over the training process, such as implementing custom loss functions, training metrics, or gradient clipping.

In [46]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Configuring the CustomTraining class to create a model with the specified vocabulary size, embedding dimension, and RNN units. The model will be used for training a text classification task.

In [47]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

**Explanation:** Configures the compilation settings for the CustomTraining model instance model. It specifies the optimizer and loss function to be used during the training process.
1. Optimizer: The optimizer = tf.keras.optimizers.Adam() statement sets the optimizer to the Adam optimization algorithm, a popular choice for deep learning models.
2. Loss Function: The loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) statement sets the loss function to Sparse Categorical Crossentropy. This loss function is suitable for text classification tasks where the target labels are represented as integer IDs. The from_logits=True argument indicates that the predictions from the model are treated as logits, which are raw output values before applying any activation function.

In [48]:
model.fit(dataset, epochs=1) # trains the CustomTraining model instance model using the given dataset for one epoch

172/172 [==============================] - 17s 63ms/step - loss: 2.7266


In [49]:
EPOCHS = 10 # create your own custom training loop

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1572
Epoch 1 Batch 50 Loss 2.0791
Epoch 1 Batch 100 Loss 1.9337
Epoch 1 Batch 150 Loss 1.9029

Epoch 1 Loss: 1.9893
Time taken for 1 epoch 14.59 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8136
Epoch 2 Batch 50 Loss 1.7358
Epoch 2 Batch 100 Loss 1.6979
Epoch 2 Batch 150 Loss 1.5958

Epoch 2 Loss: 1.7106
Time taken for 1 epoch 13.30 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5958
Epoch 3 Batch 50 Loss 1.5647
Epoch 3 Batch 100 Loss 1.5535
Epoch 3 Batch 150 Loss 1.4915

Epoch 3 Loss: 1.5504
Time taken for 1 epoch 12.10 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5170
Epoch 4 Batch 50 Loss 1.4313
Epoch 4 Batch 100 Loss 1.4394
Epoch 4 Batch 150 Loss 1.4493

Epoch 4 Loss: 1.4506
Time taken for 1 epoch 20.47 sec
_____________________________________________________________________

### **Pertanyaan**

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

**Jawab**

**Perbedaan Utama Antara Kode Tugas dan Praktikum 2**

Perbedaan utama antara kode tugas dan praktikum 2 terletak pada pendekatan pelatihan yang digunakan. Kode tugas menggunakan pendekatan khusus yang memungkinkan pengguna untuk mengendalikan pelatihan dengan tingkat kontrol yang lebih tinggi. Sementara itu, kode praktikum 2 menggunakan pendekatan yang lebih sederhana yang dikelola oleh framework TensorFlow secara otomatis.

**Pendekatan Khusus dalam Kode Tugas**

Pendekatan khusus dalam kode tugas menggunakan metode `train_step()` yang didefinisikan secara eksplisit dalam subclass `CustomTraining`. Metode ini memungkinkan pengguna untuk menentukan langkah-langkah pelatihan secara spesifik, seperti perhitungan gradien dan pembaruan bobot. Dengan demikian, pendekatan ini memberikan pengguna fleksibilitas dan kontrol yang lebih besar atas proses pelatihan.

**Pendekatan Sederhana dalam Kode Praktikum 2**

Pendekatan sederhana dalam kode praktikum 2 menggunakan metode `fit()` yang disediakan oleh framework TensorFlow. Metode ini mengelola sebagian besar detail pelatihan secara otomatis, sehingga pengguna tidak perlu mendefinisikan langkah-langkah pelatihan secara eksplisit. Pendekatan ini lebih cocok untuk tugas-tugas pelatihan standar di mana tidak perlu melakukan kustomisasi tingkat rendah yang signifikan.

**Kesimpulan**

Perbedaan antara dua pendekatan tersebut mencerminkan trade-off antara fleksibilitas dan kompleksitas kode. Pendekatan khusus memberikan fleksibilitas yang lebih besar, tetapi juga membutuhkan kode yang lebih kompleks. Pendekatan sederhana lebih mudah digunakan, tetapi memberikan kontrol yang lebih sedikit kepada pengguna.